PLEASE READ: We will be using this to visualize the output of the model implementations. The impl.py file can be used for writing the detailed implementation of the functions, or any other helper functions that we might find useful. 

Organizing some thoughts:
1. Finetuning YOLO or training our own classification model:
    a. need a database for photos
    b. 
2. Finetuning some NLP model or training our own

In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
# loading the dataset 
path = "path to the dataset"
dataset = torch.load(path)

In [ ]:
# defining some hyper parameters
batch_size = 32
train_ratio = 0.7
val_ration = 0.2
test_ratio = 0.1

# splitting the dataset
train_size = int(len(dataset) * train_ratio)
val_size = int(len(dataset) * val_ration)
test_size = len(dataset) - train_size - val_size

train_set, val_set, test_set = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

# creating the dataloader
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True)

Training Loops

In [ ]:
def val(model, val_data_loader, criterion):
  """
  Validate the model on the validation dataset.

  Inputs:
  model (torch.nn.Module): The deep learning model to be trained.
  val_data_loader (torch.utils.data.DataLoader): DataLoader for the validation dataset.
  criterion (torch.nn.Module): Loss function to compute the training loss.

  Returns:
  Validation loss (float)
  """
  val_running_loss = 0
  with torch.no_grad():
    for i, (inputs, labels) in enumerate(val_data_loader, 0):
      outputs = model(inputs)
      loss = criterion(outputs, labels)
      val_running_loss += loss.item()

  return val_running_loss

In [ ]:
def train(model, data_loader, val_data_loader, criterion, optimizer, epochs):
    """
    Train the model on the training dataset.
    Inputs:
    model (torch.nn.Module): The deep learning model to be trained.
    data_loader (torch.utils.data.DataLoader): DataLoader for the training dataset.
    val_data_loader (torch.utils.data.DataLoader): DataLoader for the validation dataset.
    criterion (torch.nn.Module): Loss function to compute the training loss.
    optimizer (torch.optim.Optimizer): Optimizer used for updating the model parameters.
    epochs (int): Number of training epochs.

    Returns:
    Tuple of (train_loss_arr, val_loss_arr), an array of the training and validation
    losses at each epoch
    """
    train_loss_arr = []
    val_loss_arr = []
    running_loss = 0.0

    for epoch in range(epochs):
        running_loss = 0.0

        for i, b in enumerate(data_loader):
          optimizer.zero_grad()
          inputs, labels = b
          outputs = model(inputs)
          loss = criterion(outputs, labels)
          running_loss += loss.item() * inputs.shape[0]
          loss.backward()
          optimizer.step()

        train_loss_arr.append(running_loss / train_size)
        val_loss_arr.append(val(model,val_data_loader,criterion))


    print('Training finished.')
    return train_loss_arr, val_loss_arr

In [ ]:
# Initializing hyperparameters for the layers
input_dim = 4
hidden_layer_1 = 128
hidden_layer_2 = 64
output_dim = 1

In [ ]:
# Defining the structure of the model
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_layer_1)
        self.fc2 = nn.Linear(hidden_layer_1, hidden_layer_2)
        self.fc3 = nn.Linear(hidden_layer_2, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))  
        return x